# Lagged raster point sampling

### Description of the datasets

We have a point dataset, in the form of a CSV file with lat/lon columns. 

Each row in the dataset represents a household as surveyed at a point in time, given by reference_month and reference_location. (In this instance the data are extracted from DHS household surveys.)

Each household has a location (lat/lon) but many households are at the same location (for anonymity); the location is known as a cluster and is unique by surveyid and cluster_number. 

Most but not necessarily all households at the same location (cluster) in a survey will be surveyed in the same month.

We have a folder of rasters which represent monthly data for a single variable of interest (i.e. one raster per month). These are named such that the filename contains the month in the form `*YYYY.MM*` i.e. 4 digits then a dot then 2 digits.

### Description of the task

For each household we want to generate a time series of raster values for the 12 months prior to the survey date, and the survey date. That is, we need to sample 13 rasters, corresponding to the reference month/year, and each of the 12 preceding months, and eventually output a table that is like the input with 13 additional columns for the values at month 0, month -1, month -2 etc.



In [148]:
import pandas as pd
import numpy as np

### Sample of the data
In these first few rows the households (hhid) within the cluster are all sampled in the same month but this isn't always true.
Note that the hhid is as taken from the source data and consists of numbers and spaces - the values should not be .trim()ed as this could break joining downstream.

In [2]:
df = pd.read_csv('peak_urban_ssa_hh_coords_ET-fix.csv', 
                 usecols=['surveyid','cluster_number','hhid','reference_month','reference_year','lat','lon'])
df.head()

,surveyid,cluster_number,hhid,reference_month,reference_year,lat,lon
0,211,1,117,10,2001,10.84476,2.109562
1,211,1,1 1,10,2001,10.84476,2.109562
2,211,1,1 2,10,2001,10.84476,2.109562
3,211,1,1 3,10,2001,10.84476,2.109562
4,211,1,1 4,10,2001,10.84476,2.109562


In [25]:
df.dtypes

surveyid             int64
cluster_number       int64
hhid                object
reference_month      int64
reference_year       int64
lat                float64
lon                float64
dtype: object

### Generating the sample dates for each row

For passing to the raster sampler we will ultimately need a long-format table (stacked), one row for each location / hh / lag combination, i.e. in the case of 12 months lag, 13 rows for each input row.

It seems strangely hard to define a way in pandas to apply a function that converts each row into multiple rows.

Instead we will add the lags to each row (wide format). 

In [3]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [4]:
N_MONTHS = 12
ONE_MONTH = relativedelta(months=1)
        
def add_lags_to_row(row):
    yr = row['reference_year']
    mth = row['reference_month']
    feat_date = date(yr, mth, 1)
    new_rows = []
    for i in range(N_MONTHS + 1):
        lag_date = feat_date - i * ONE_MONTH
        lag_y = lag_date.year
        lag_m = lag_date.month
        row[f'lag_n{i}'] = i
        row[f'lag_yr{i}'] = lag_y
        row[f'lag_m{i}'] = lag_m
    return row

In [77]:
df_test = df.head(20)

In [79]:
wide_df_test = df_test.apply(add_lags_to_row, axis=1)
wide_df_test['id'] = wide_df_test.index
wide_df_test.head()

,surveyid,cluster_number,hhid,reference_month,reference_year,lat,lon,lag_n0,lag_yr0,lag_m0,...,lag_n10,lag_yr10,lag_m10,lag_n11,lag_yr11,lag_m11,lag_n12,lag_yr12,lag_m12,id
0,211,1,117,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,0
1,211,1,1 1,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,1
2,211,1,1 2,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,2
3,211,1,1 3,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,3
4,211,1,1 4,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,4


This is pretty slow due to all those lookups and python loops on each row. So we'll parallelise it. 

Tried dask which I've not used before; didn't quite get there, would need to redefine the mapped function to expect series I think


In [59]:
import dask.dataframe as dd
from dask.multiprocessing import get

In [ ]:
ddata = dd.from_pandas(df, npartitions=100)
#df.compute(get=dask.threaded.get, num_workers=20)
def apply_lagger_to_DF(df): return df.apply((lambda row: add_lags_to_row(**row)), axis=1)
wide_df = ddata.map_partitions(apply_lagger_to_DF).compute(get=get, num_workers=25)

Pandarallel is much more straightforward for this case. 

Install the jupyter widgets extension for progress bars if necessary (https://github.com/nalepae/pandarallel) first

In [237]:
from pandarallel import pandarallel

Limit workers to 25 so as not to take over server (72 cores); can use memory fs as server has large allocation

In [6]:
pandarallel.initialize(nb_workers=25, progress_bar=True, use_memory_fs=True)

INFO: Pandarallel will run on 25 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
wide_df  = df.parallel_apply(add_lags_to_row, axis=1)

Still takes a while, so checkpoint:

In [10]:
wide_df.to_csv('peak_urban_hh_lags_wide_all.csv')

In [27]:
wide_df.to_parquet('peak_urban_hh_lags_wide_all', compression='GZIP')

In [13]:
wide_df['id'] = wide_df.index

In [14]:
wide_df.head()

,surveyid,cluster_number,hhid,reference_month,reference_year,lat,lon,lag_n0,lag_yr0,lag_m0,...,lag_n10,lag_yr10,lag_m10,lag_n11,lag_yr11,lag_m11,lag_n12,lag_yr12,lag_m12,id
0,211,1,117,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,0
1,211,1,1 1,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,1
2,211,1,1 2,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,2
3,211,1,1 3,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,3
4,211,1,1 4,10,2001,10.84476,2.109562,0,2001,10,...,10,2000,12,11,2000,11,12,2000,10,4


Use the super-handy wide_to_long function to stack / pseudo-normalise this output to have one row per unique combination of location, time (raster) and how many lag months this time is for this household.



In [16]:
melted = pd.wide_to_long(wide_df, ['lag_n','lag_m', 'lag_yr'], i=['id'], j='thing')
melted.head(5)

,,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_n,lag_m,lag_yr
id,thing,,,,,,,,,,
0,0,1,211,10.84476,2.109562,117,2001,10,0,10,2001
1,0,1,211,10.84476,2.109562,1 1,2001,10,0,10,2001
2,0,1,211,10.84476,2.109562,1 2,2001,10,0,10,2001
3,0,1,211,10.84476,2.109562,1 3,2001,10,0,10,2001
4,0,1,211,10.84476,2.109562,1 4,2001,10,0,10,2001


This will have duplicates where multiple households in the same location (cluster) are interviewed in the same month (which is the norm).

We could use this df directly for the raster sampling, but it'll be more efficient to get rid of the duplicates in this dimension to cut down raster sampling (maybe).

In [19]:
extract_pts = melted.drop_duplicates(subset=['cluster_number', 'surveyid', 'lag_n', 'lag_yr', 'lag_m', 'lag_yr'])
extract_pts.head()

,,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_n,lag_m,lag_yr
id,thing,,,,,,,,,,
0,0,1,211,10.844760,2.109562,117,2001,10,0,10,2001
17,0,2,211,10.685410,1.074763,2 8,2001,7,0,7,2001
21,0,2,211,10.685410,1.074763,2 4,2001,8,0,8,2001
34,0,3,211,10.613593,1.273059,3 2,2001,8,0,8,2001
46,0,4,211,10.512361,0.945930,4 1,2001,8,0,8,2001


We will use rasterio to handle the sampling

In [39]:
import rasterio as rio
from rasterio import RasterioIOError

Define a function that will take a dataframe or rather sub-set of one with a common month and year for extraction, and extract the matching raster for all these point locations.
The month and year will be taken from the first row of the group and we're not currently checking they are the same across the passed data. Data path is hardcoded here for now

In [238]:
def extract_raster_vals(grp):
    coords = list(zip(grp['lon'], grp['lat']))
    firstrow = grp.iloc[0]
    req_yr = str(firstrow['lag_yr'])
    req_m = str(firstrow['lag_m']).zfill(2)
    #rastername = f'no2/temis_omi_no2.{req_yr}.{req_m}.tif'
    rastername = f'no2/temis_omi_no2.{req_yr}.{req_m}.tif'
    print(f"Trying {rastername} for {grp.shape[0]} points")
    #print(coords)
    try:
        with rio.open(rastername) as src:
            grp['rasterval'] = [x[0] for x in src.sample(coords, masked=True)]
    except RasterioIOError:
        grp['rasterval'] = ''
    return grp

Use groupby to call the function for each yr/mth subset of the data in turn

In [226]:
sampled = extract_pts.loc[pd.notnull(extract_pts.lat)].groupby(['lag_m','lag_yr']).apply(extract_raster_vals)

Trying no2/temis_omi_no2.2001.01.tif for 276 points
Trying no2/temis_omi_no2.2005.01.tif for 1145 points
Trying no2/temis_omi_no2.2006.01.tif for 1113 points
Trying no2/temis_omi_no2.2007.01.tif for 1112 points
Trying no2/temis_omi_no2.2008.01.tif for 2666 points
Trying no2/temis_omi_no2.2009.01.tif for 332 points
Trying no2/temis_omi_no2.2010.01.tif for 3216 points
Trying no2/temis_omi_no2.2011.01.tif for 2367 points
Trying no2/temis_omi_no2.2012.01.tif for 1916 points
Trying no2/temis_omi_no2.2013.01.tif for 2687 points
Trying no2/temis_omi_no2.2014.01.tif for 4461 points
Trying no2/temis_omi_no2.2015.01.tif for 3901 points
Trying no2/temis_omi_no2.2016.01.tif for 4018 points
Trying no2/temis_omi_no2.2017.01.tif for 849 points
Trying no2/temis_omi_no2.2018.01.tif for 3768 points
Trying no2/temis_omi_no2.2019.01.tif for 3 points
Trying no2/temis_omi_no2.2001.02.tif for 276 points
Trying no2/temis_omi_no2.2005.02.tif for 1154 points
Trying no2/temis_omi_no2.2006.02.tif for 1290 points


In [227]:
sampled

,,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_n,lag_m,lag_yr,rasterval
id,thing,,,,,,,,,,,
0,0,1,211,10.844760,2.109562,117,2001,10,0,10,2001,
17,0,2,211,10.685410,1.074763,2 8,2001,7,0,7,2001,
21,0,2,211,10.685410,1.074763,2 4,2001,8,0,8,2001,
34,0,3,211,10.613593,1.273059,3 2,2001,8,0,8,2001,
46,0,4,211,10.512361,0.945930,4 1,2001,8,0,8,2001,
...,...,...,...,...,...,...,...,...,...,...,...,...
699744,12,541,542,-15.416081,28.370067,541 2,2018,8,12,8,2017,221
699769,12,542,542,-11.150453,32.946955,542 1,2018,9,12,9,2017,263
699794,12,543,542,-15.769918,28.299570,543 26,2018,11,12,11,2017,93


merge (left join) the extracted data back onto the with-duplicates inputs and remove the duplicate columns caused by the merge (pandas keeps both even when names match and are tested equal in the join)

In [228]:
full_results_long = pd.merge(melted, sampled, how='left', on=['cluster_number', 'surveyid', 'lag_n', 'lag_m', 'lag_yr'], 
                             indicator=True, validate='m:1',
                            suffixes=('','_y'))

full_results_long.drop(full_results_long.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [215]:
full_results_long.head()

,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_n,lag_m,lag_yr,rasterval,_merge
0,1,211,10.84476,2.109562,117,2001,10,0,10,2001,0.271354,both
1,1,211,10.84476,2.109562,1 1,2001,10,0,10,2001,0.271354,both
2,1,211,10.84476,2.109562,1 2,2001,10,0,10,2001,0.271354,both
3,1,211,10.84476,2.109562,1 3,2001,10,0,10,2001,0.271354,both
4,1,211,10.84476,2.109562,1 4,2001,10,0,10,2001,0.271354,both


check that any surveys which didn't get through the raster extraction are the ones we expect (the ones with null lat column, which got dropped just befor sampling)

In [229]:
full_results_long[full_results_long['_merge']!= 'both']['surveyid'].unique()

array([253])

In [230]:
np.unique(full_results_long[full_results_long['rasterval']==''][['surveyid', 'reference_year', 'reference_month']], axis=0)


array([[ 211, 2001,    6],
       [ 211, 2001,    7],
       [ 211, 2001,    8],
       [ 211, 2001,    9],
       [ 211, 2001,   10],
       [ 248, 2005,    3],
       [ 248, 2005,    4],
       [ 248, 2005,    5],
       [ 248, 2005,    6],
       [ 248, 2005,    7],
       [ 248, 2005,    8],
       [ 260, 2005,    7],
       [ 260, 2005,    8],
       [ 260, 2005,    9],
       [ 491, 2018,    1],
       [ 491, 2018,    2],
       [ 511, 2018,    5],
       [ 511, 2018,    6],
       [ 511, 2018,    7],
       [ 511, 2018,    8],
       [ 511, 2018,    9],
       [ 511, 2018,   10],
       [ 511, 2018,   11],
       [ 511, 2018,   12],
       [ 517, 2018,    7],
       [ 517, 2018,    8],
       [ 517, 2018,    9],
       [ 517, 2018,   10],
       [ 528, 2018,    8],
       [ 528, 2018,    9],
       [ 528, 2018,   10],
       [ 528, 2018,   11],
       [ 528, 2018,   12],
       [ 539, 2018,    3],
       [ 539, 2018,    4],
       [ 539, 2018,    5],
       [ 542, 2018,    7],
 

In [172]:
full_results_test = full_results_long.head(250000).copy()
full_results_test

,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_n,lag_m,lag_yr,rasterval,_merge
0,1,211,10.844760,2.109562,117,2001,10,0,10,2001,,both
1,1,211,10.844760,2.109562,1 1,2001,10,0,10,2001,,both
2,1,211,10.844760,2.109562,1 2,2001,10,0,10,2001,,both
3,1,211,10.844760,2.109562,1 3,2001,10,0,10,2001,,both
4,1,211,10.844760,2.109562,1 4,2001,10,0,10,2001,,both
...,...,...,...,...,...,...,...,...,...,...,...,...
249995,70,363,-22.614343,17.069834,7016,2013,5,0,5,2013,31,both
249996,70,363,-22.614343,17.069834,70 7,2013,5,0,5,2013,31,both
249997,70,363,-22.614343,17.069834,70 5,2013,5,0,5,2013,31,both
249998,70,363,-22.614343,17.069834,70 8,2013,5,0,5,2013,31,both


Now pivot it back to one set of 3 columns for each lag period i.e. (year,month,value) of extracted raster for each lag amount

https://stackoverflow.com/a/55252414/4150190

There isn't a long_to_wide like there is a wide_to_long!

In [231]:
full_results_long['idx'] = full_results_long.groupby(['cluster_number','surveyid', 'lat', 'lon', 'hhid', 'reference_year','reference_month' ]).cumcount()

In [232]:
pivoted = full_results_long.pivot_table(index=['cluster_number','surveyid', 'lat', 'lon', 'hhid', 'reference_year','reference_month' ],
                                       columns='idx',
                                     values=['lag_n', 'lag_m', 'lag_yr', 'rasterval'],
                       aggfunc='first')
pivoted
#melted_test.pivot(index=['surveyid', 'cluster_number', 'hhid', 'reference_month','reference_year', 'lat', 'lon'], 
#                 columns=['lag_n'], values=['lag_yr', 'lag_m'])

lag_m  \
idx                                                                                      0    
cluster_number surveyid lat      lon      hhid         reference_year reference_month         
1              211      10.84476 2.109562          1 1 2001           10                 10   
                                                   1 2 2001           10                 10   
                                                   1 3 2001           10                 10   
                                                   1 4 2001           10                 10   
                                                   1 5 2001           10                 10   
...                                                                                     ...   
2029           266      0.00000  0.000000      2029135 2006           9                   9   
                                               2029139 2006           9                   9   
                                               2029141 2006           9                   9   
                                               2029149 2006           9                   9   
                                               2029152 2006           9                   9   

                                                                                          \
idx                                                                                   1    
cluster_number surveyid lat      lon      hhid         reference_year reference_month      
1              211      10.84476 2.109562          1 1 2001           10               9   
                                                   1 2 2001           10               9   
                                                   1 3 2001           10               9   
                                                   1 4 2001           10               9   
                                                   1 5 2001           10               9   
...                                                                                   ..   
2029           266      0.00000  0.000000      2029135 2006           9                8   
                                               2029139 2006           9                8   
                                               2029141 2006           9                8   
                                               2029149 2006           9                8   
                                               2029152 2006           9                8   

                                                                                          \
idx                                                                                   2    
cluster_number surveyid lat      lon      hhid         reference_year reference_month      
1              211      10.84476 2.109562          1 1 2001           10               8   
                                                   1 2 2001           10               8   
                                                   1 3 2001           10               8   
                                                   1 4 2001           10               8   
                                                   1 5 2001           10               8   
...                                                                                   ..   
2029           266      0.00000  0.000000      2029135 2006           9                7   
                                               2029139 2006           9                7   
                                               2029141 2006           9                7   
                                               2029149 2006           9                7   
                                               2029152 2006           9                7   

                                                                                          \
idx                                                                                  

In [233]:
pivoted = pivoted.sort_index(axis=1, level=1)
pivoted

lag_m  \
idx                                                                                      0    
cluster_number surveyid lat      lon      hhid         reference_year reference_month         
1              211      10.84476 2.109562          1 1 2001           10                 10   
                                                   1 2 2001           10                 10   
                                                   1 3 2001           10                 10   
                                                   1 4 2001           10                 10   
                                                   1 5 2001           10                 10   
...                                                                                     ...   
2029           266      0.00000  0.000000      2029135 2006           9                   9   
                                               2029139 2006           9                   9   
                                               2029141 2006           9                   9   
                                               2029149 2006           9                   9   
                                               2029152 2006           9                   9   

                                                                                      lag_n  \
idx                                                                                      0    
cluster_number surveyid lat      lon      hhid         reference_year reference_month         
1              211      10.84476 2.109562          1 1 2001           10                  0   
                                                   1 2 2001           10                  0   
                                                   1 3 2001           10                  0   
                                                   1 4 2001           10                  0   
                                                   1 5 2001           10                  0   
...                                                                                     ...   
2029           266      0.00000  0.000000      2029135 2006           9                   0   
                                               2029139 2006           9                   0   
                                               2029141 2006           9                   0   
                                               2029149 2006           9                   0   
                                               2029152 2006           9                   0   

                                                                                      lag_yr  \
idx                                                                                       0    
cluster_number surveyid lat      lon      hhid         reference_year reference_month          
1              211      10.84476 2.109562          1 1 2001           10                2001   
                                                   1 2 2001           10                2001   
                                                   1 3 2001           10                2001   
                                                   1 4 2001           10                2001   
                                                   1 5 2001           10                2001   
...                                                                                      ...   
2029           266      0.00000  0.000000      2029135 2006           9                 2006   
                                               2029139 2006           9                 2006   
                                               2029141 2006           9                 2006   
                                               2029149 2006           9                 2006   
                                               2029152 2006           9                 2006   

                                                                               

In [234]:
pivoted.columns

MultiIndex([(    'lag_m',  0),
            (    'lag_n',  0),
            (   'lag_yr',  0),
            ('rasterval',  0),
            (    'lag_m',  1),
            (    'lag_n',  1),
            (   'lag_yr',  1),
            ('rasterval',  1),
            (    'lag_m',  2),
            (    'lag_n',  2),
            (   'lag_yr',  2),
            ('rasterval',  2),
            (    'lag_m',  3),
            (    'lag_n',  3),
            (   'lag_yr',  3),
            ('rasterval',  3),
            (    'lag_m',  4),
            (    'lag_n',  4),
            (   'lag_yr',  4),
            ('rasterval',  4),
            (    'lag_m',  5),
            (    'lag_n',  5),
            (   'lag_yr',  5),
            ('rasterval',  5),
            (    'lag_m',  6),
            (    'lag_n',  6),
            (   'lag_yr',  6),
            ('rasterval',  6),
            (    'lag_m',  7),
            (    'lag_n',  7),
            (   'lag_yr',  7),
            ('rasterval',  7),
        

In [235]:
pivoted.columns = [f'{x}_{y}' for x,y in pivoted.columns]
pivoted = pivoted.reset_index()
pivoted

,cluster_number,surveyid,lat,lon,hhid,reference_year,reference_month,lag_m_0,lag_n_0,lag_yr_0,...,lag_yr_10,rasterval_10,lag_m_11,lag_n_11,lag_yr_11,rasterval_11,lag_m_12,lag_n_12,lag_yr_12,rasterval_12
0,1,211,10.84476,2.109562,1 1,2001,10,10,0,2001,...,2000,,11,11,2000,,10,12,2000,
1,1,211,10.84476,2.109562,1 2,2001,10,10,0,2001,...,2000,,11,11,2000,,10,12,2000,
2,1,211,10.84476,2.109562,1 3,2001,10,10,0,2001,...,2000,,11,11,2000,,10,12,2000,
3,1,211,10.84476,2.109562,1 4,2001,10,10,0,2001,...,2000,,11,11,2000,,10,12,2000,
4,1,211,10.84476,2.109562,1 5,2001,10,10,0,2001,...,2000,,11,11,2000,,10,12,2000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689552,2029,266,0.00000,0.000000,2029135,2006,9,9,0,2006,...,2005,67,10,11,2005,30,9,12,2005,39
689553,2029,266,0.00000,0.000000,2029139,2006,9,9,0,2006,...,2005,67,10,11,2005,30,9,12,2005,39
689554,2029,266,0.00000,0.000000,2029141,2006,9,9,0,2006,...,2005,67,10,11,2005,30,9,12,2005,39
689555,2029,266,0.00000,0.000000,2029149,2006,9,9,0,2006,...,2005,67,10,11,2005,30,9,12,2005,39


This gives our final output; save it to csv.
Optionally delete the `lag_n_*`, `lag_m_*`  and `lag_yr_*` columns as once we've checked the logic they are redundant

In [236]:
pivoted.to_csv('test_no2_output.csv', index=False)